In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [10]:
import random

import numpy as np
import pandas as pd

from src.models.gcn import GCN_var_2layer
from src.models.gat import GAT

from src.models.train_model import train_with_loss
from src.models.train_model import random_splits

from src.models.reg import make_preg_ce_ce
from src.models.reg import make_preg_ce_ce_alt
from src.models.reg import make_lap_loss_ce

from src.models.reg import compute_a_hat

from src.models.evaluate_model import evaluate0
from src.models.evaluate_model import evaluate1

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from sklearn.manifold import TSNE

from src.models.evaluate_model import test
from tqdm import tqdm

from notebooks.FinalAnalysisAbdul.random_split import random_split as abdul_random_splits


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset_cora = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
dataset_citeseer = Planetoid(root='data/Planetoid', name='CiteSeer', transform=T.NormalizeFeatures())
dataset_pubmed = Planetoid(root='data/Planetoid', name='PubMed', transform=T.NormalizeFeatures())

"""data = dataset[0].to(device)
data = random_splits(data, 20, 30)
data.reg_mask = torch.ones_like(data.train_mask, dtype=torch.bool)
A_hat = compute_a_hat(data)"""



'data = dataset[0].to(device)\ndata = random_splits(data, 20, 30)\ndata.reg_mask = torch.ones_like(data.train_mask, dtype=torch.bool)\nA_hat = compute_a_hat(data)'

In [12]:
for dataset in [dataset_citeseer, dataset_pubmed]:
    for seed in [0,1,2,3,4]:
        for model_name in ['gat_preg', 'gcn_vanilla', 'gcn_preg', 'gat_vanilla']:
            torch.manual_seed(seed)
            random.seed(seed)

            data = dataset[0].to(device)
            data = abdul_random_splits(data, 20, 30, seed=seed)
            data.reg_mask = torch.ones_like(data.train_mask, dtype=torch.bool)
            A_hat = compute_a_hat(data)

            if model_name == 'gcn_vanilla':
                loss_fn = make_preg_ce_ce_alt(0, A_hat)    
                model = GCN_var_2layer(hidden_channels=64)

            elif model_name == 'gcn_preg':
                loss_fn = make_preg_ce_ce_alt(0.7, A_hat)    
                model = GCN_var_2layer(hidden_channels=64)

            elif model_name == 'gat_vanilla':
                loss_fn = make_preg_ce_ce_alt(0, A_hat)    
                model = GAT(dataset)

            elif model_name == 'gat_preg':
                loss_fn = make_preg_ce_ce_alt(0.6, A_hat)    
                model = GAT(dataset)

            else:
                raise Exception
            model = train_with_loss(model, data, loss_fn, num_epochs=200)

            acc = evaluate0(model, data)

            train_acc, val_acc, test_acc = evaluate1(model, data)
            metrics.append({'model_name': model_name, 'seed': seed,  'train_acc': np.round(train_acc,4), 'val_acc': np.round(val_acc,4), 'test_acc': np.round(test_acc,4)})
            print(metrics[-1])
            # print(f'mu: {mu}, train_acc: {train_acc:.4f}, val_acc: {val_acc:.4f}, test_acc: {test_acc:.4f}')
            # print('-------------------------------------------------------------')


KeyError: 0

In [ ]:
metrics_1 = []
for dataset in [dataset_cora]:
    for seed in [0,1,2,3,4]:
        for model_name in ['gcn_vanilla', 'gcn_preg']:
            torch.manual_seed(seed)
            random.seed(seed)

            data = dataset[0].to(device)
            data = random_splits(data, 20, 30)
            data.reg_mask = torch.ones_like(data.train_mask, dtype=torch.bool)
            A_hat = compute_a_hat(data)

            if model_name == 'gcn_vanilla':
                loss_fn = make_preg_ce_ce_alt(0, A_hat)    
                model = GCN_var_2layer(hidden_channels=64)

            elif model_name == 'gcn_preg':
                loss_fn = make_preg_ce_ce_alt(0.7, A_hat)    
                model = GCN_var_2layer(hidden_channels=64)

            elif model_name == 'gat_vanilla':
                loss_fn = make_preg_ce_ce_alt(0, A_hat)    
                model = GAT(dataset)

            elif model_name == 'gat_preg':
                loss_fn = make_preg_ce_ce_alt(0.6, A_hat)    
                model = GAT(dataset)

            else:
                raise Exception
            model = train_with_loss(model, data, loss_fn, num_epochs=200)

            acc = evaluate0(model, data)

            train_acc, val_acc, test_acc = evaluate1(model, data)
            metrics_1.append({'model_name': model_name, 'seed': seed,  'train_acc': np.round(train_acc,4), 'val_acc': np.round(val_acc,4), 'test_acc': np.round(test_acc,4)})
            print(metrics_1[-1])
            # print(f'mu: {mu}, train_acc: {train_acc:.4f}, val_acc: {val_acc:.4f}, test_acc: {test_acc:.4f}')
            # print('-------------------------------------------------------------')
